In [1]:
# Part II - VacationPy
# Now let's use your skills in working with weather data to plan future vacations. 
    #Use jupyter-gmaps and the Google Places API for this part of the assignment.

# Note: Remember that any API usage beyond the $200 credit will be charged to your personal account. 
    #You can set quotas and limits to your daily requests to be sure you can't be charged. Check out Google Maps Platform Billing and Manage your cost of use for more information.

# Note: if you having trouble displaying the maps, try running jupyter nbextension enable --py gmaps 
    #in your environment and retry.

# To complete this part of the assignment,you will need to do the following:

# Create a heat map that displays the humidity for every city from Part I.

# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned 
    #by your API requests to a reasonable number.

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# As final considerations:

# You must complete your analysis using a Jupyter notebook.
# You must use the Matplotlib or Pandas plotting libraries.
# For Part I, you must include a written description of three observable trends based on the data.
# For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
# You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
# For max intensity in the heat map, try setting it to the highest humidity found in the data set.


# Hints and Considerations

# The city data you generate is based on random coordinates as well as different query times. 
    #As such, your outputs will not be an exact match to the provided starter notebook.

# If you'd like a refresher on the geographic coordinate system, this site has great information.

# Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, 
    #you should be able to answer basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

# A starter code for Citipy has been provided. However, if you're craving an extra challenge, 
    #push yourself to learn how it works: citipy Python library. Before you try to incorporate the library 
    #into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

# Part of our expectation in this challenge is that you will use critical thinking skills to understand how 
    #and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

# In building your script, pay attention to the cities you are using in your query pool. 
    #Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 
#     #500 cities concentrated in one region of the world? Even if you were a geographic genius, 
#     simply rattling 500 cities based on your human selection would create a biased dataset. 
#     Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

# Once you have computed the linear regression for one chart, the process will be similar for all others.
    #As a bonus, try to create a function that will create these charts based on different parameters.

# Remember that each coordinate will trigger a separate call to the Google API. If you're creating your own 
    #criteria to plan your vacation, try to reduce the results in your DataFrame to 10 or fewer cities.

# Ensure your repository has regular commits (i.e. 20+ commits) and a thorough README.md file.

# Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, 
    #then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

In [2]:
!pip install gmaps

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json
import gmaps
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

%matplotlib inline
# Import API key
from api_keys import g_key

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
!jupyter nbextension list

Known nbextensions:
  config dir: C:\Users\Sue\.jupyter\nbconfig
    notebook section
      jupyter-gmaps/extension enabled 
  config dir: C:\Users\Sue\anaconda3\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 


Config option `kernel_spec_manager_class` not recognized by `ListNBExtensionsApp`.
      - Validating: ok
      - Validating: ok


In [5]:
# Store Part I results into DataFrame

# Load the csv exported in Part I to a DataFrame

file = pd.read_csv("../weather_py/Weather_Dataframe_cities.csv")
print(file.shape)
file=file.dropna()
print(file.shape)
file.head()

 

(670, 10)
(606, 10)


,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,naron,ES,2021-01-13 15:27:10,43.5167,-8.1528,75.0,81.0,48.272,1.54
1,1,albany,US,2021-01-13 15:16:16,42.6001,-73.9662,97.0,63.0,36.266,0.62
2,2,codrington,AU,2021-01-13 15:20:45,-38.2667,141.9667,1.0,92.0,51.872,2.56
3,3,butaritari,KI,2021-01-13 15:16:18,3.0707,172.7902,72.0,76.0,81.842,9.52
4,4,puerto escondido,MX,2021-01-13 15:27:10,15.8500,-97.0667,40.0,74.0,82.670,2.06


In [14]:
gmaps.configure(api_key=g_key)
fig=gmaps.figure()

heatmap_layer=gmaps.heatmap_layer(file[["Latitude", "Longitude"]], weights=file["Humidity (%)"], dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create new DataFrame fitting weather criteria

# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
temp_file=file.loc[(file['Max Temp (F)']>=70)&(file['Max Temp (F)']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed (mph)']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness (%)']==0)]
ideal_weather.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
13,13,cape town,ZA,2021-01-13 15:25:11,-33.9258,18.4232,0.0,64.0,75.272,3.09
31,31,chuy,UY,2021-01-13 15:16:51,-33.6971,-53.4616,0.0,46.0,71.024,5.09
78,78,abu zabad,SD,2021-01-13 15:27:16,12.3500,29.2500,0.0,23.0,70.700,4.66
104,104,salalah,OM,2021-01-13 15:16:25,17.0151,54.0924,0.0,25.0,75.470,4.63
117,117,geraldton,AU,2021-01-13 15:20:38,-28.7667,114.6000,0.0,64.0,70.070,4.63


In [8]:
# Hotel Map

# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
# Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df    

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL90M6N7nWqjgDzJQaioz512wXcabQa6qg9lNHeHhJGY4EsWC0PdiQ_JG47maWVNeQP1MNJw-TSKydptVGtyFxMojyHJbnKxBPe10detAk2fsqVFycGaLgQaeB5TBFdOuLB4Ee23sOByzEjfY1sDuloWd-GEndDlHomxIp5NODFysMzdi_j7D29I_D7QZpNGHu7-4IbQZGjNP_Mj1PFf_KUAWzUtDWga2pXJ1KU2C0G6NmOSw9XhuhogARz3-WAJwTQd4x1BzFe_jihClhgG-3cWGY27h8b1kdk4nnsQjBR66zXDbxApXiXzzVQ3bRrHaY-xSesRSouvmvXlsQDm_1MOEW0ApxzgHYRVHGpFh5FOQmXB5yBsrySBTmDEs8HMiZM3CbsqJj_v7g8BOXBu_DBCy7tInOv1s3yn32qCr_PpeZ3VE1-Vz_IPXRh-cyNcbfO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.9596092,
                    "lng": 18.3852981
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.95831972010728,
                        "lng": 18.38671007989272
                    },
                    "southwest": {
                        "lat": -33.96

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.6881437,
                    "lng": -53.4465252
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.68670522010728,
                        "lng": -53.44534982010727
                    },
                    "southwest": {
                        "lat": -33.68940487989273,
                        "lng": -53.44804947989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bertelli Chu\u00ed Hotel",
            "photos": [
                {
                    "height": 2268,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/117258703733313923022\"

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.0049368,
                    "lng": 54.0708943
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.00630417989272,
                        "lng": 54.07224222989272
                    },
                    "southwest": {
                        "lat": 17.00360452010727,
                        "lng": 54.06954257010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Star Emirates garden villas",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwK5OFRrcbqykgubaB3I2JMpaJrZ48vLCuOePruredR3D3BzWGNu78AmEdl3e66mpo16EsPndqdHe-OKyX73zQLqIzWOO2gkAKhjAteYHWrp-9_GNUdsVR-T-fMyXn-FUIrKGE9tx6NQWX4Tzkl2JqPzXLBHiAH7XBg4P9evo2iM9pqxmMyg0Hbt1KGRIg3AjohBjxU8YugKr_QNUi6wBSwVV1U-v-bEJsts90MtUWJn1T6LZ8x-CS1P-lAMOs8LKUGbEnB--pRSj_sQqXJMlEe0Bif7x5eVBs5IKVdZObcXAeRGTJBG3GQzv1Sa_B4HJYqRB7bU8O-UPxqjmYY09NGeHg6Z5kvJXj_xRopOMredwAyge8x_3lAYq9uCjP9LsxZw-1j_UzRkjHv71BwpGTEcfY6uAFgU0zwSjgeZYMvUaXlNUC7IpR7xR-T-LAFtkc6m",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.7738192,
                    "lng": 114.6082717
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.77237427010728,
                        "lng": 114.6095486298927
                    },
                    "southwest": {
                        "lat": -28.7

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.7098697,
                    "lng": -76.20645979999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.70853152010728,
                        "lng": -76.20460952010728
                    },
                    "southwest": {
                        "lat": -13.71123117989272,
                        "lng": -76.20730917989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hostal Mandala",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 594,
                    "html_attributions": [
                    

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.0325904,
                    "lng": 37.2570057
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.03390552989272,
                        "lng": 37.25838357989272
                    },
                    "southwest": {
                        "lat": 25.03120587010728,
                        "lng": 37.25568392010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "HP Red Sea Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/110396888368178619339\">baligh moharm

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.9680397,
                    "lng": -114.0170643
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.96913807989272,
                        "lng": -114.0156800701073
                    },
                    "southwest": {
                        "lat": 27.96643842010728,
                        "lng": -114.0183797298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel TerraSal",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
          

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.0076092,
                    "lng": 17.9458689
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.00632052010727,
                        "lng": 17.94702232989272
                    },
                    "southwest": {
                        "lat": -33.00902017989272,
                        "lng": 17.94432267010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hoedjiesbaai Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 593,
                    "html_attributions": [
                        "<

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.


,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,13,cape town,ZA,2021-01-13 15:25:11,-33.9258,18.4232,0.0,64.0,75.272,3.09,Atlanticview Cape Town Boutique Hotel
1,31,chuy,UY,2021-01-13 15:16:51,-33.6971,-53.4616,0.0,46.0,71.024,5.09,Bertelli Chuí Hotel
2,78,abu zabad,SD,2021-01-13 15:27:16,12.3500,29.2500,0.0,23.0,70.700,4.66,
3,104,salalah,OM,2021-01-13 15:16:25,17.0151,54.0924,0.0,25.0,75.470,4.63,Star Emirates garden villas
4,117,geraldton,AU,2021-01-13 15:20:38,-28.7667,114.6000,0.0,64.0,70.070,4.63,Ocean Centre Hotel
5,147,pisco,PE,2021-01-13 15:20:07,-13.7000,-76.2167,0.0,65.0,77.270,9.77,Hostal Mandala
6,163,port blair,IN,2021-01-13 15:16:23,11.6667,92.7500,0.0,77.0,77.432,4.95,"Symphony Samudra, Port Blair"
7,196,alice springs,AU,2021-01-13 15:20:49,-23.7000,133.8833,0.0,49.0,77.270,1.54,DoubleTree by Hilton Hotel Alice Springs
8,258,bumbuna,SL,2021-01-13 15:27:32,9.0500,-11.7333,0.0,50.0,79.052,1.28,
9,272,niono,ML,2021-01-13 15:27:33,14.2526,-5.9930,0.0,15.0,78.728,4.38,Sanoussi Baco


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])


fig.add_layer(markers)


In [11]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))